# Imports

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import os, sys, subprocess, time
import plotly.express as px
import plotly,re

# Colab setup ------------------
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade plotly"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE) 

    cmd = "git clone https://github.com/glensk/Crypto-ETL-Pipeline" 
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE) 
    time.sleep(2) # sleep for 2 second since colab needs time to create files
    !cd Crypto-ETL-Pipeline && python setup.py install

import glassnode_utils as gu
import defillama_utils as du
import mongodb_utils as mu

protocols_defilama  = ['curve', 'makerdao', 'aave', 'wbtc', 'compound', 'lido', 'sushiswap', 'yearn-finance', 'uniswap']
protocols_glassnode = [ 'CRV', 'MKR', 'AAVE', 'WBTC', 'COMP', 'LDO', 'SUSHI', 'YFI', 'UNI']
tokens = list(zip(protocols_defilama, protocols_glassnode))

def find_substr_in_list(str="mc_*",liste=[]):
    r = re.compile(str)
    return list(filter(r.match, liste))

def get_subdataframe(dall=None,str=None):
    a=find_substr_in_list(str=str,liste=dall.columns)
    a.insert(0,'date')
    return dall[a]

# Import: Marketcap & Price & TVL

In [3]:
# make a dataframe for the marketcap (mca)
print('getting marketcap...')
dall = gu.marketcap("CRV").date
for token in tokens:
    mc=gu.marketcap(token[1])
    
    # for outlyer in wbtc
    mc=mc.loc[mc[mc.columns[1]] <= 10e+15]
    
    dall=pd.merge(dall,mc, how='outer', on='date')
dall=dall.sort_values(by='date')

print('getting price...')
for token in tokens:
    dall=pd.merge(dall,gu.price(token[1]), how='outer', on='date')
dall=dall.sort_values(by='date')

print('getting tvl...')
for token in tokens:
    dall=pd.merge(dall,du.tvl(token[0]), how='outer', on='date')
dall=dall.sort_values(by='date')

getting marketcap...
getting price...
getting tvl...


# Analyze imported data

In [7]:
# plot of markecap
if False:
    df = get_subdataframe(dall,str="mc_*")
    df = df[(df['date'] > '2020-09-01')]
    fig = px.line(df, x=df.date, y=df.columns[1:],title="MarketCap in USD",labels={'x':'date','value':'MarketCap USD'})
    fig.show()

if False:
    # plot of price
    df = get_subdataframe(dall,str="price_*")
    df = df[(df['date'] > '2020-09-01')]
    fig = px.line(df, x=df.date, y=df.columns[1:],title="Price in USD",labels={'x':'date','value':'Price USD'})
    fig.show()

if False:
    # plot of TVL
    df = get_subdataframe(dall,str="tvl_*")
    df = df[(df['date'] > '2020-09-01')]
    fig = px.line(df, x=df.date, y=df.columns[1:],title="TVL in USD",labels={'x':'date','value':'TVL USD'})
    fig.show()

# Write to & read from MongoDB

In [20]:
#mu.write_db(dall)
#dall = mu.read_db() # resets dall dataframe

written...


# Visualization 1: TVL as a share of MarketCap

In [21]:
df = pd.DataFrame(dall.date)
for token in tokens:
    label = token[1].lower()
    df[label]=dall['tvl_'+label]/dall['mc_'+label]
    #df[label]=dall['mc_'+label]/dall['tvl_'+label]
    df['log_'+label]=np.log(df[label])

df = df[(df['date'] > '2020-09-01')]
df = get_subdataframe(df,str="log_*") # plot only log
fig = px.line(df, x=df.date, y=df.columns[1:],title="Visualization 1: (Log of) TVL as a share of MarketCap",labels={'value':'log(TVL / MarketCap)'})
fig.show()

/Users/mac/opt/miniconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning:

divide by zero encountered in log



# Visualization 2: Relationship between TVL and price information

In [22]:
df = pd.DataFrame(dall.date)
for token in tokens:
    label = token[1].lower()
    df[label]=dall['tvl_'+label]/dall['price_'+label]
    df['log_'+label]=np.log(df[label])


df = df[(df['date'] > '2020-09-01')]
df = get_subdataframe(df,str="log_*") # plot only log
fig = px.line(df, x=df.date, y=df.columns[1:],title="Visualization 2: (Log of) Relationship between TVL and price information",
    labels={'value':'Log( TVL / Price )'})
fig.show()

/Users/mac/opt/miniconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning:

divide by zero encountered in log



# Questions to address if more time:
* Compare TVL from defillama to TVL from glassnode. 
* Would it make sense to separate TVL by the tokens that are locked (in the respective protocols)? (and to sum those up over all protocols to get something as illiquid supply of this token?)
* TVL could depend on vesting schedule of protocol.


# Actionable insights:
* given only TVL as a metric for valuation, CRV and Lido seem most attractive.
* Sharp changes in TVL seem to be often followed by significant price jumps. (see Visualization 3, where peaks seem to correlate with price jumps -> this needs to be further quantified when more time.)

# Visualization 3

In [23]:
df = pd.DataFrame(dall.date)
for token in tokens:
    label = token[1].lower()
    df['price_'+label] = dall['price_'+label]
    df['tvl_'+label] = dall['tvl_'+label]
    df['mc_'+label] = dall['mc_'+label]
    if True: # apply moving average
        avg=2
        df['price_ma_'+label] = df['price_'+label].rolling(window=avg).mean()
        df['tvl_ma_'+label] = df['tvl_'+label].rolling(window=avg).mean()
        df['mc_ma_'+label] = df['mc_'+label].rolling(window=avg).mean()

    df['price_ma_diff_'+label] = df['price_ma_'+label].diff()
    df['tvl_ma_diff_'+label] = df['tvl_ma_'+label].diff()
    df['mc_ma_diff_'+label] = df['mc_ma_'+label].diff()
    
    df['D_'+label] = df['tvl_ma_diff_'+label]/df['price_ma_diff_'+label]
        

df = get_subdataframe(df,str="D_*") # plot moving average
df = df[(df['date'] > '2020-09-01')]
fig = px.line(df, x=df.date, y=df.columns[1:],title="Visualization 3: Derivative: dTVL/dPrice (on 2d moving averages)",
    labels={'value':'dTVL / dPrice'})
fig.show()